### SVM　パラメータチューニング

In [1]:
import numpy as np
import pandas as pd

from sklearn import svm
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split, GridSearchCV


In [2]:
# pickle ファイルから読み込み
train_pkl = pd.read_pickle('./pd_train.pk2')

In [3]:
train_pkl.shape

(891, 15)

In [4]:
# Age, Fare は削除
# ダミー変数の先頭は削除
train_pkl.drop(['Age', 'Fare', 'Sex_female', 'Embarked_C', 'Pclass_1'], inplace=True, axis=1)

In [5]:
# 学習が終わらないので、特徴量を上位５つにする
train_pkl.drop(['Parch', 'Embarked_S', 'Embarked_Q'], inplace=True, axis=1)

In [6]:
train_pkl['Age_bin'] = train_pkl['Age_bin'].astype('float16')

In [7]:
train_pkl['Fare_bin'] = train_pkl['Fare_bin'].astype('float16')

In [8]:
train_pkl['SibSp'] = train_pkl['SibSp'].astype('int8')

In [9]:
train_pkl['Sex_male'] = train_pkl['Sex_male'].astype('bool')

In [10]:
train_pkl['Pclass_2'] = train_pkl['Pclass_2'].astype('bool')

In [11]:
train_pkl['Pclass_3'] = train_pkl['Pclass_3'].astype('bool')

In [12]:
train_pkl.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   SibSp     891 non-null    int8   
 2   Age_bin   891 non-null    float16
 3   Fare_bin  891 non-null    float16
 4   Sex_male  891 non-null    bool   
 5   Pclass_2  891 non-null    bool   
 6   Pclass_3  891 non-null    bool   
dtypes: bool(3), float16(2), int64(1), int8(1)
memory usage: 14.0 KB


In [13]:
train_pkl.describe()

,Survived,SibSp,Age_bin,Fare_bin
count,891.000000,891.000000,891.000000,891.000000
mean,0.383838,0.523008,2.419922,0.904785
std,0.486592,1.102743,1.356445,1.980469
min,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,2.000000,0.000000
50%,0.000000,0.000000,2.000000,0.000000
75%,1.000000,1.000000,3.000000,1.000000
max,1.000000,8.000000,8.000000,20.000000


In [14]:
# 訓練とテストデータに分割
train, test = train_test_split(train_pkl, test_size=0.2, random_state=42)

In [15]:
# ターゲットと特徴量の分割
train_X = train.iloc[:, 1:].values
train_y = train.Survived.values

### 検証するパラメータの設定

In [16]:
grid_param = [
    {'C': [0.001, 0.01, 0.1, 1, 5, 10, 50, 100, 1000],
    'kernel': ['linear', 'rbf'],
    'gamma': [.0001, .001, .01, .1, .5, 1, 10, 30],
#     'shrinking': [True, False],
    'random_state': [42]},
    {'C': [0.001, 0.01, 0.1, 1, 10],
    'gamma': [.0001, .001, .01,.1,.2,.5,1,10],
    'random_state': [42],
    'kernel':['poly']}
]

### グリッドサーチ

In [18]:
gs = GridSearchCV(estimator=svm.SVC(grid_param), param_grid=grid_param, scoring='accuracy', cv=5, return_train_score=False)

In [ ]:
gs.fit(train_X, train_y)

### 一番良かったパラメータで学習

In [ ]:
gs.best_score_

In [ ]:
gs.best_params_

In [ ]:
# ターゲットと特徴量の分割
train_X = train.iloc[:, 1:].values
train_y = train.Survived.values

In [ ]:
SVC = svm.SVC(**gs.best_params_)

In [ ]:
SVC = SVC.fit(train_X, train_y)

### テストデータで予測

In [ ]:
# ターゲットと特徴量の分割
test_x = test.iloc[:, 1:].values
test_y = test.Survived.values

In [ ]:
test_x.shape, test_y.shape

In [ ]:
pred_y = SVC.predict(test_x)

In [ ]:
confusion_matrix(test_y, pred_y)

In [ ]:
accuracy_score(test_y, pred_y)

### 検証データで予測

In [ ]:
# 検証データ読み込み
valid = pd.read_pickle('./pd_test.pk2')

In [ ]:
valid.shape

In [ ]:
# ID の保存
valid_pass = valid.PassengerId.values

In [ ]:
valid_X = valid.iloc[:, 1:]

In [ ]:
valid_X.drop(['Age', 'Fare', 'Sex_female', 'Embarked_C', 'Pclass_1'], inplace=True, axis=1)

In [ ]:
# 学習が終わらないので、特徴量を上位５つにする
valid_X.drop(['Parch', 'Embarked_S', 'Embarked_Q'], inplace=True, axis=1)

In [ ]:
valid_X['Age_bin'] = valid_X['Age_bin'].astype('float16')

In [ ]:
valid_X['Fare_bin'] = valid_X['Fare_bin'].astype('float16')

In [ ]:
valid_X['SibSp'] = valid_X['SibSp'].astype('int8')

In [ ]:
valid_X['Sex_male'] = valid_X['Sex_male'].astype('bool')

In [ ]:
valid_X['Pclass_2'] = valid_X['Pclass_2'].astype('bool')

In [ ]:
valid_X['Pclass_3'] = valid_X['Pclass_3'].astype('bool')

In [ ]:
valid_X.info()

In [ ]:
valid_X.shape, train_X.shape

In [ ]:
pred_valid_y = SVC.predict(valid_X)

In [ ]:
pred_valid_y.shape

In [ ]:
type(valid_pass), type(pred_valid_y)

In [ ]:
result_df = pd.DataFrame(pred_valid_y, valid_pass, columns=['Survived'])

In [ ]:
result_df.to_csv("./SVC_3.csv", index_label='PassengerId')